In [15]:
import pandas as pd

In [16]:
MODEL_COLUMN = "model"
BASELINE="BEVfusion"
CORRUPTIONS = ["beamsreducing", "brightness", "dark", "fog", "missingcamera", "motionblur", "pointsreducing", "snow", "spatialmisalignment", "temporalmisalignment"]
NICE_NAMES = {
    "beamsreducing": "Beams Red.",
    "brightness": "Brightness",
    "dark": "Darkness",
    "fog":"Fog",
    "missingcamera": "Missing Cam.",
    "motionblur": "Motion Blur",
    "pointsreducing": "Points Red.",
    "snow": "Snow",
    "spatialmisalignment": "Spatial Mis.",
    "temporalmisalignment": "Temporal Mis."
}
MODELS_TO_LINKS = {
    "CMT": "https://github.com/junjie18/CMT",
    "DeepInteraction": "https://github.com/fudan-zvg/DeepInteraction",
    "TransFusion": "https://github.com/XuyangBai/TransFusion",
    "Sparsefusion": "https://github.com/yichen928/SparseFusion",
    "BEVfusion": "https://github.com/mit-han-lab/bevfusion",
    "IS-Fusion": "https://github.com/yinjunbo/IS-Fusion",
    "UniBEV_cat": "https://github.com/tudelft-iv/UniBEV",
    "UniBEV_avg": "https://github.com/tudelft-iv/UniBEV",
    "UniBEV_cnw": "https://github.com/tudelft-iv/UniBEV",
    "MEFormer": "https://github.com/hanchaa/MEFormer",
    "MEFormer w/o PME": "https://github.com/hanchaa/MEFormer",
    "MoME": "https://github.com/konyul/MoME"
}
MODELS = list(MODELS_TO_LINKS.keys())
MODELS_TO_LINKS_MD = {k: f"[{k}]({v})" for k, v in MODELS_TO_LINKS.items()}
SEVERITY_LEVELS = ["1", "2", "3"]

In [17]:
df = pd.read_csv("../evaluation/evaluations_nds.csv")

In [18]:
df = df.dropna()

In [19]:
df

,model,clean,temporalmisalignment_s1,temporalmisalignment_s2,temporalmisalignment_s3,spatialmisalignment_s1,spatialmisalignment_s2,spatialmisalignment_s3,pointsreducing_s1,pointsreducing_s2,...,dark_s3,brightness_s1,brightness_s2,brightness_s3,beamsreducing_s1,beamsreducing_s2,beamsreducing_s3,snow_s1,snow_s2,snow_s3
0,CMT,0.7290,0.6572,0.5680,0.4988,0.6462,0.5860,0.5367,0.7012,0.6834,...,0.6786,0.7038,0.6852,0.6605,0.6826,0.5873,0.4497,0.6483,0.6190,0.5554
1,DeepInteraction,0.6909,0.6139,0.5237,0.4544,0.5863,0.5014,0.4267,0.6522,0.6242,...,0.6254,0.6785,0.6719,0.6574,0.6162,0.4571,0.2839,0.5526,0.5311,0.4896
2,TransFusion,0.7084,0.6306,0.5437,0.4766,0.5708,0.4761,0.4095,0.6554,0.6190,...,0.6978,0.7056,0.7041,0.7007,0.6222,0.4591,0.2636,0.5447,0.5353,0.5087
3,Sparsefusion,0.7315,0.6523,0.5622,0.4901,0.6061,0.5149,0.4461,0.6881,0.6578,...,0.6978,0.7205,0.7146,0.7039,0.6593,0.5139,0.3380,0.5859,0.5704,0.5337
4,BEVfusion,0.7144,0.6341,0.5471,0.4085,0.5903,0.4943,0.4266,0.6684,0.6379,...,0.6861,0.6967,0.6913,0.6851,0.6396,0.4899,0.3199,0.5782,0.5572,0.5233
5,IS-Fusion,0.7365,0.6537,0.5609,0.4892,0.6221,0.5214,0.4359,0.6827,0.6489,...,0.6925,0.7198,0.7091,0.6917,0.6694,0.5068,0.3266,0.5619,0.5463,0.5115
6,UniBEV_cat,0.6782,0.6097,0.5283,0.4644,0.5813,0.5126,0.4390,0.6470,0.6263,...,0.6461,0.6717,0.6629,0.6493,0.6141,0.5101,0.4199,0.5811,0.5292,0.4390
7,UniBEV_avg,0.6842,0.6147,0.5330,0.4677,0.5801,0.5117,0.4619,0.6530,0.6313,...,0.6501,0.6757,0.6658,0.6537,0.6183,0.5151,0.4264,0.5849,0.5269,0.4323
8,UniBEV_cnw,0.6853,0.6122,0.5292,0.4646,0.5794,0.5065,0.4492,0.6514,0.6295,...,0.6518,0.6764,0.6670,0.6524,0.6181,0.5061,0.3981,0.5798,0.5229,0.4224
9,MEFormer,0.7385,0.6659,0.5762,0.5013,0.6494,0.5866,0.5274,0.7078,0.6863,...,0.6919,0.7143,0.6982,0.6788,0.6885,0.5837,0.4201,0.6622,0.6288,0.5565


## Resistance Ability (RA)

In [20]:
for c in CORRUPTIONS:
    df[f"RA_{c}_s1"] = df[c+"_s1"] / df['clean']
    df[f"RA_{c}_s2"] = df[c+"_s2"] / df['clean']
    df[f"RA_{c}_s3"] = df[c+"_s3"] / df['clean']
    df[f"RA_{c}"] = (df[f"RA_{c}_s1"] + df[f"RA_{c}_s2"] + df[f"RA_{c}_s3"]) / 3
    
df["mRA"] = df[[f"RA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [21]:
df[[MODEL_COLUMN]+[f"RA_{c}" for c in CORRUPTIONS] + ["mRA"]]

,model,RA_beamsreducing,RA_brightness,RA_dark,RA_fog,RA_missingcamera,RA_motionblur,RA_pointsreducing,RA_snow,RA_spatialmisalignment,RA_temporalmisalignment,mRA
0,CMT,0.786283,0.937128,0.948377,0.806356,0.973845,0.841198,0.924829,0.833425,0.808825,0.788294,0.864856
1,DeepInteraction,0.654798,0.968688,0.929367,0.583008,0.841994,0.831765,0.882038,0.759058,0.730641,0.768080,0.794944
2,TransFusion,0.632835,0.993036,0.988095,0.754329,0.985460,0.825758,0.850508,0.747553,0.685300,0.776821,0.823970
3,Sparsefusion,0.688631,0.974710,0.963272,0.767100,0.953930,0.847801,0.879335,0.770107,0.714103,0.776760,0.833575
4,BEVfusion,0.676278,0.967292,0.968692,0.752146,0.973871,0.865668,0.872107,0.773936,0.705114,0.741741,0.829685
5,IS-Fusion,0.680154,0.959765,0.951527,0.757954,0.953338,0.873229,0.860240,0.733062,0.714822,0.771125,0.825522
6,UniBEV_cat,0.758921,0.975081,0.966529,0.790721,0.970068,0.797847,0.909663,0.761476,0.753416,0.787575,0.847130
7,UniBEV_avg,0.759914,0.972035,0.964971,0.775699,0.964289,0.797428,0.909286,0.752265,0.756942,0.787002,0.843983
8,UniBEV_cnw,0.740454,0.970767,0.965563,0.775427,0.956856,0.784036,0.905248,0.741816,0.746680,0.781166,0.836801
9,MEFormer,0.763846,0.943940,0.951659,0.790070,0.940510,0.834845,0.918438,0.833898,0.795938,0.786910,0.856005


### RA Latex

In [22]:
rn = {f"RA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({MODEL_COLUMN: "Model"})
df_latex = df[[MODEL_COLUMN]+[f"RA_{c}" for c in CORRUPTIONS] + ["mRA"]].rename(columns=rn)
df_latex = df_latex.sort_values("mRA", ascending=False)
df_latex = df_latex.style.format(precision=3).hide().highlight_max(axis=0, props="textbf:--rwrap;")
print(df_latex.to_latex(hrules=True))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
Model & Beams Red. & Brightness & Darkness & Fog & Missing Cam. & Motion Blur & Points Red. & Snow & Spatial Mis. & Temporal Mis. & mRA \\
\midrule
CMT & \textbf{0.786} & 0.937 & 0.948 & \textbf{0.806} & 0.974 & 0.841 & \textbf{0.925} & 0.833 & \textbf{0.809} & \textbf{0.788} & \textbf{0.865} \\
MEFormer & 0.764 & 0.944 & 0.952 & 0.790 & 0.941 & 0.835 & 0.918 & \textbf{0.834} & 0.796 & 0.787 & 0.856 \\
MoME & 0.756 & 0.943 & 0.948 & 0.788 & 0.962 & 0.835 & 0.919 & 0.833 & 0.788 & 0.786 & 0.856 \\
MEFormer w/o PME & 0.755 & 0.945 & 0.954 & 0.789 & 0.937 & 0.834 & 0.917 & 0.831 & 0.786 & 0.785 & 0.853 \\
UniBEV_cat & 0.759 & 0.975 & 0.967 & 0.791 & 0.970 & 0.798 & 0.910 & 0.761 & 0.753 & 0.788 & 0.847 \\
UniBEV_avg & 0.760 & 0.972 & 0.965 & 0.776 & 0.964 & 0.797 & 0.909 & 0.752 & 0.757 & 0.787 & 0.844 \\
\textbf{UniBEV_cnw} & 0.740 & 0.971 & 0.966 & 0.775 & 0.957 & 0.784 & 0.905 & 0.742 & 0.747 & 0.781 & 0.837 \\
Sparsefusion & 0.689 & 0.975 & 0.963

### RA Markdown

In [23]:
rn = {f"RA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({"model": "Model"})
rn.update({"clean": "Clean"})
df_ra = df.sort_values("mRA", ascending=False)
df_ra = df_ra[[MODEL_COLUMN]+["clean"]+ ["mRA"] + [f"RA_{c}" for c in CORRUPTIONS]].rename(columns=rn)
df_ra = df_ra.sort_values("mRA", ascending=False)

def to_bold_float(cell):
    try:
        return f'**{float(cell):.3f}**'
    except (ValueError, TypeError):
        return cell
    
def to_float(cell):
    try:
        return f'{float(cell):.3f}'
    except (ValueError, TypeError):
        return cell

def md_highlight_max(row):
    try:
        is_max = row == row.max()
        return [to_bold_float(cell) if max_cell else to_float(cell) for cell, max_cell in zip(row, is_max)]
    except (ValueError, TypeError):
        return row

df_markdown = df_ra.apply(md_highlight_max, axis=0).replace(MODELS_TO_LINKS_MD)
print(df_markdown.to_markdown(index=False))

| Model                                                           | Clean     | mRA       | Beams Red.   | Brightness   | Darkness   | Fog       | Missing Cam.   | Motion Blur   | Points Red.   | Snow      | Spatial Mis.   | Temporal Mis.   |
|:----------------------------------------------------------------|:----------|:----------|:-------------|:-------------|:-----------|:----------|:---------------|:--------------|:--------------|:----------|:---------------|:----------------|
| [CMT](https://github.com/junjie18/CMT)                          | 0.729     | **0.865** | **0.786**    | 0.937        | 0.948      | **0.806** | 0.974          | 0.841         | **0.925**     | 0.833     | **0.809**      | **0.788**       |
| [MEFormer](https://github.com/hanchaa/MEFormer)                 | **0.739** | 0.856     | 0.764        | 0.944        | 0.952      | 0.790     | 0.941          | 0.835         | 0.918         | **0.834** | 0.796          | 0.787           |
| [MoME](https://github.com/

### RA HTML

In [24]:
def html_highlight_max(s): 
    if s.dtype == object: 
        is_max = [False for _ in range(s.shape[0])] 
    else: 
        is_max = s == s.max() 
    return ['background: grey; font-weight: bold' if cell else '' for cell in is_max] 


df_ra.style.hide().format(precision=3).apply(html_highlight_max)
# print(df_markdown.style.hide_index().format(precision=3).apply(html_highlight_max).to_html())

Model,Clean,mRA,Beams Red.,Brightness,Darkness,Fog,Missing Cam.,Motion Blur,Points Red.,Snow,Spatial Mis.,Temporal Mis.
CMT,0.729,0.865,0.786,0.937,0.948,0.806,0.974,0.841,0.925,0.833,0.809,0.788
MEFormer,0.739,0.856,0.764,0.944,0.952,0.790,0.941,0.835,0.918,0.834,0.796,0.787
MoME,0.735,0.856,0.756,0.943,0.948,0.788,0.962,0.835,0.919,0.833,0.788,0.786
MEFormer w/o PME,0.737,0.853,0.755,0.945,0.954,0.789,0.937,0.834,0.917,0.831,0.786,0.785
UniBEV_cat,0.678,0.847,0.759,0.975,0.967,0.791,0.970,0.798,0.910,0.761,0.753,0.788
UniBEV_avg,0.684,0.844,0.760,0.972,0.965,0.776,0.964,0.797,0.909,0.752,0.757,0.787
UniBEV_cnw,0.685,0.837,0.740,0.971,0.966,0.775,0.957,0.784,0.905,0.742,0.747,0.781
Sparsefusion,0.732,0.834,0.689,0.975,0.963,0.767,0.954,0.848,0.879,0.770,0.714,0.777
BEVfusion,0.714,0.830,0.676,0.967,0.969,0.752,0.974,0.866,0.872,0.774,0.705,0.742
IS-Fusion,0.737,0.826,0.680,0.960,0.952,0.758,0.953,0.873,0.860,0.733,0.715,0.771


# Relative Resistance Ability (RRA)

In [25]:
for c in CORRUPTIONS:
    non_baseline = df.loc[df[MODEL_COLUMN] != BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1)
    baseline = df.loc[df[MODEL_COLUMN] == BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1).iloc[0]
    df[f"RRA_{c}"] = (((non_baseline / baseline) - 1)) * 100
    df.loc[df[MODEL_COLUMN] == BASELINE, f"RRA_{c}"] = 0
    
df["mRRA"] = df[[f"RRA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [26]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]]

,model,clean,RRA_beamsreducing,RRA_brightness,RRA_dark,RRA_fog,RRA_missingcamera,RRA_motionblur,RRA_pointsreducing,RRA_snow,RRA_spatialmisalignment,RRA_temporalmisalignment,mRRA
0,CMT,0.7290,18.642197,-1.138392,-0.096334,9.398263,2.041012,-0.840834,8.212509,9.887261,17.052673,8.448135,7.160649
1,DeepInteraction,0.6909,-6.361253,-3.149872,-7.215452,-25.037221,-16.385588,-7.077023,-2.188219,-5.148610,0.211752,0.144681,-7.220680
2,TransFusion,0.7084,-7.209880,1.799238,1.146380,-0.552109,0.340169,-5.411524,-3.295704,-4.220172,-3.626257,3.849783,-1.718008
3,Sparsefusion,0.7315,4.263833,3.178814,1.820722,4.429280,0.297049,0.280278,3.242202,1.887020,3.699047,7.227779,3.032602
4,BEVfusion,0.7144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,IS-Fusion,0.7365,3.684283,2.291255,1.266798,3.889578,0.919893,3.993963,1.690653,-2.351239,4.512970,7.177455,2.707561
6,UniBEV_cat,0.6782,6.533738,-4.302735,-5.279129,-0.198511,-5.437907,-12.504716,-0.979081,-6.595527,1.435945,0.798893,-2.652903
7,UniBEV_avg,0.6842,7.616945,-3.757658,-4.595154,-1.228288,-5.169605,-11.777071,-0.144455,-6.909025,2.812335,1.616657,-2.153532
8,UniBEV_cnw,0.6853,5.029667,-3.728715,-4.383219,-1.104218,-5.749329,-13.119172,-0.428013,-8.054501,1.581525,1.025351,-2.893062
9,MEFormer,0.7385,16.758659,0.877912,1.555802,8.585608,-0.167689,-0.307228,8.865229,11.382408,16.688724,9.668491,7.390792


#### RRA LateX

In [27]:
rn = {f"RRA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({MODEL_COLUMN:"Model"})
rn.update({"clean": "Clean"})
df_latex = df[[MODEL_COLUMN]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]].rename(columns=rn)
df_latex = df_latex.sort_values("mRRA", ascending=False)
df_latex = df_latex.style.format(precision=3).hide().highlight_max(axis=0, props="textbf:--rwrap;")
print(df_latex.to_latex(hrules=True))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
Model & Beams Red. & Brightness & Darkness & Fog & Missing Cam. & Motion Blur & Points Red. & Snow & Spatial Mis. & Temporal Mis. & mRRA \\
\midrule
MEFormer & 16.759 & 0.878 & 1.556 & 8.586 & -0.168 & -0.307 & \textbf{8.865} & \textbf{11.382} & 16.689 & \textbf{9.668} & \textbf{7.391} \\
CMT & \textbf{18.642} & -1.138 & -0.096 & \textbf{9.398} & \textbf{2.041} & -0.841 & 8.213 & 9.887 & \textbf{17.053} & 8.448 & 7.161 \\
MoME & 15.061 & 0.400 & 0.694 & 7.891 & 1.634 & -0.685 & 8.421 & 10.852 & 14.975 & 9.128 & 6.837 \\
MEFormer w/o PME & 15.172 & 0.714 & 1.522 & 8.176 & -0.791 & -0.679 & 8.384 & 10.749 & 14.968 & 9.184 & 6.740 \\
Sparsefusion & 4.264 & \textbf{3.179} & \textbf{1.821} & 4.429 & 0.297 & 0.280 & 3.242 & 1.887 & 3.699 & 7.228 & 3.033 \\
IS-Fusion & 3.684 & 2.291 & 1.267 & 3.890 & 0.920 & \textbf{3.994} & 1.691 & -2.351 & 4.513 & 7.177 & 2.708 \\
BEVfusion & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000

### RRA Markdown

In [28]:
df_markdown = df.copy()
index_to_top = df_markdown.index[df_markdown[MODEL_COLUMN] == BASELINE].to_list()[0]
idx = [index_to_top] + [i for i in df_markdown.index if i!=index_to_top]
df_markdown = df_markdown.loc[idx]
df_markdown = df_markdown[[MODEL_COLUMN]+["clean"]+["mRRA"]+[f"RRA_{c}" for c in CORRUPTIONS]].rename(columns=rn)
df_markdown = df_markdown.sort_values("mRRA", ascending=False)

df_markdown = df_markdown.apply(md_highlight_max, axis=0).replace(MODELS_TO_LINKS_MD)
print(df_markdown.to_markdown(index=False))

| Model                                                           | Clean     | mRRA      | Beams Red.   | Brightness   | Darkness   | Fog       | Missing Cam.   | Motion Blur   | Points Red.   | Snow       | Spatial Mis.   | Temporal Mis.   |
|:----------------------------------------------------------------|:----------|:----------|:-------------|:-------------|:-----------|:----------|:---------------|:--------------|:--------------|:-----------|:---------------|:----------------|
| [MEFormer](https://github.com/hanchaa/MEFormer)                 | **0.739** | **7.391** | 16.759       | 0.878        | 1.556      | 8.586     | -0.168         | -0.307        | **8.865**     | **11.382** | 16.689         | **9.668**       |
| [CMT](https://github.com/junjie18/CMT)                          | 0.729     | 7.161     | **18.642**   | -1.138       | -0.096     | **9.398** | **2.041**      | -0.841        | 8.213         | 9.887      | **17.053**     | 8.448           |
| [MoME](https://github.